# Graves' Greenery — Setup
---
## SQL via DuckDB (ipython-sql + PrettyTable)
Single DuckDB connection (no server). All CSVs in `/data` are exposed as VIEWS (or TABLES) using the active %sql connection.

In [ ]:
# --- CONFIG ---
REPO_USER = "danielsgraves"
REPO_NAME = "Graves_Greenery_Analysis"                  # repo folder name after clone
REPO_DIR  = f"/content/{REPO_NAME}"
DATA_DIR  = f"{REPO_DIR}/data"                           # CSVs live here
DB_FILE   = f"{REPO_DIR}/outputs/graves_greenery.duckdb"  # DuckDB file
LOAD_AS_VIEWS = True   # True: CSV -> VIEW (zero copy). False: CSV -> TABLE (materialize)
PRINT_SAMPLE_ROWS = 5   # preview rows in dashboard (0 to disable)

In [ ]:
# --- SYNC REPO: clone if missing, else pull latest ---
import os, subprocess

def run(cmd):
    p = subprocess.run(cmd, shell=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    print(p.stdout)
    if p.returncode != 0:
        raise RuntimeError(f"Command failed: {cmd}")

if not os.path.exists(REPO_DIR):
    run(f"git clone https://github.com/{REPO_USER}/{REPO_NAME}.git {REPO_DIR}")
else:
    os.chdir(REPO_DIR)
    run("git fetch --all --prune")
    run("git pull --rebase")

os.makedirs(f"{REPO_DIR}/outputs", exist_ok=True)
os.chdir(REPO_DIR)
print("Working dir:", os.getcwd())
run("ls -la")

In [ ]:
# --- INSTALL: ipython-sql + PrettyTable + DuckDB SQLAlchemy driver ---
%pip -q install ipython-sql prettytable duckdb duckdb-engine sqlalchemy pandas

# Load SQL magic and configure rendering
%reload_ext sql
%config SqlMagic.autopandas = False   # PrettyTable output (set True for DataFrame output)
%config SqlMagic.feedback = False

# Choose a valid PrettyTable style (fallback to DataFrame if none)
import prettytable as pt
candidates = ['MARKDOWN','PLAIN_COLUMNS','MSWORD_FRIENDLY','DOUBLE_BORDER','SINGLE_BORDER','DEFAULT']
avail = [s for s in candidates if hasattr(pt, s)]
if avail:
    chosen = avail[0]
    get_ipython().run_line_magic('config', f"SqlMagic.style = '{chosen}'")
    print(f"PrettyTable style set to: {chosen}")
else:
    get_ipython().run_line_magic('config', "SqlMagic.autopandas = True")
    print("No PrettyTable styles detected; switched to DataFrame rendering.")

In [ ]:
# --- RESET & CONNECT TO DUCKDB SAFELY ---
%reload_ext sql
try:
    from sql.connection import Connection
    for key in list(Connection.connections.keys()):
        try:
            Connection.close(key)
            print('Closed:', key)
        except Exception as e:
            print('Skip:', key, '->', e)
    print('All old connections closed.')
except Exception as e:
    print('No prior connections or cleanup error:', e)

db_url  = f"duckdb:///{DB_FILE}"
%sql $db_url
%config SqlMagic.autopandas = False  # keep PrettyTable as default
print('✅ Connected to:', db_url)

In [ ]:
# --- LOAD ALL CSVs USING THE ACTIVE %sql CONNECTION (NO SECOND DUCKDB CONNECTION) ---
import os, glob
from IPython import get_ipython

files = sorted(glob.glob(os.path.join(DATA_DIR, "*.csv")))
print(f"Found {len(files)} CSV(s) in {DATA_DIR}")
if not files:
    print("⚠️ No CSVs found. Put your data files under /data in the repo.")
else:
    for path in files:
        table = os.path.splitext(os.path.basename(path))[0]
        if LOAD_AS_VIEWS:
            sql = f"""
            CREATE OR REPLACE VIEW {table} AS
            SELECT * FROM read_csv_auto('{path}', HEADER=TRUE);
            """
        else:
            sql = f"""
            CREATE OR REPLACE TABLE {table} AS
            SELECT * FROM read_csv_auto('{path}', HEADER=TRUE);
            """
        get_ipython().run_cell_magic('sql', '', sql)
        print(("VIEW ready: " if LOAD_AS_VIEWS else "TABLE loaded: ") + table)

print("Done.")

In [ ]:
%%sql
SHOW TABLES;

In [ ]:
# --- ROW COUNT DASHBOARD (ROBUST, AUTOPANDAS-AWARE) ---
%config SqlMagic.autopandas = True   # return Pandas DataFrames directly from %sql
import pandas as pd

def _to_df(res):
    # If SqlMagic.autopandas=True, res is already a DataFrame; otherwise has .DataFrame()
    return res if isinstance(res, pd.DataFrame) else (res.DataFrame() if res is not None else pd.DataFrame())

# 1) Get table names (prefer duckdb_tables(); fallback to SHOW TABLES)
tables = []
try:
    res = %sql SELECT table_name FROM duckdb_tables() WHERE NOT internal ORDER BY table_name;
    df_names = _to_df(res)
    if not df_names.empty and 'table_name' in df_names.columns:
        tables = df_names['table_name'].tolist()
    else:
        raise ValueError('duckdb_tables() did not return table_name column')
except Exception as e:
    print('Fallback to SHOW TABLES due to error:', e)
    res2 = %sql SHOW TABLES;
    df2 = _to_df(res2)
    if not df2.empty:
        col = 'name' if 'name' in df2.columns else df2.columns[0]
        tables = df2[col].tolist()

rows = []
for t in tables:
    cnt = %sql SELECT COUNT(*) AS n FROM $t;
    df_cnt = _to_df(cnt)
    n = int(df_cnt.iloc[0, 0]) if not df_cnt.empty else 0
    rows.append({'table': t, 'rows': n})

counts = pd.DataFrame(rows).sort_values('table').reset_index(drop=True)
display(counts)

if PRINT_SAMPLE_ROWS and tables:
    for t in tables:
        print(f"\n### {t} (first {PRINT_SAMPLE_ROWS} rows)")
        _ = %sql SELECT * FROM $t LIMIT $PRINT_SAMPLE_ROWS;

%config SqlMagic.autopandas = False   # switch back to PrettyTable for later cells

## SQL Sandbox
Use `%%sql` below to run queries directly against DuckDB.

In [ ]:
%%sql
SELECT * FROM dim_customers LIMIT 5;
